## Random Forrest & Decision Forrest

In [ ]:
## imports 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [183]:
## dictionary for the dataset name & paths 
DATASETS ={
    "small":"../Data/small/glass.data",
    "medium":"../Data/medium/drug_consumption.data",
    "large":"../Data/large/c2k_data_comma.csv"
               }

## Data Split Parameters 
TRAIN_SIZE = 0.8
TEST_SIZE = 1 - TRAIN_SIZE

## Random Forest Parameters
NUM_TREES = 100 ## 1,10,25,50,75,100 
NUM_FEATURES = 10  ## 1,3, int(log_2(M)+1), sqrt(M), where M is the number of features
RANDOM_STATE = 42

In [223]:
def load_dataset(dataset_name):
    """
    Loads the dataset from the dictionary.
    """
    if dataset_name == 'small':
        ## exclude the first column ID (labeled 0 to 10, so use 1-10)
        df = pd.read_csv(DATASETS[dataset_name],header=None)
        df.drop(df.columns[0], axis=1, inplace=True)
        df.columns = list(range(0,10))
        return df
    
def split_train_test(data: pd.DataFrame, train_size: float) -> pd.DataFrame:
    """
    Splits the data into training and testing sets.
    """
    train_data = data.sample(frac=train_size, random_state=RANDOM_STATE)
    test_data = data.drop(train_data.index)
    return train_data, test_data

nb_train = int(np.floor(0.9 * len(df)))
df = df.sample(frac=1, random_state=217)
X_train = df[features][:nb_train]
y_train = df['Survived'][:nb_train].values
X_test = df[features][nb_train:]
y_test = df['Survived'][nb_train:].values

In [243]:
## load the data 
small = load_dataset("small")
TARGET_COL = small.columns[-1]
## split the data 
train, test = split_train_test(small, TRAIN_SIZE)
## split the train into features and labels
nb_train = int(np.floor(0.9 * len(train)))
X_train, y_train = train.drop(TARGET_COL, axis=1)[:nb_train], train[TARGET_COL][:nb_train]
X_test, y_test = test.drop(TARGET_COL, axis=1)[nb_train:], test[TARGET_COL][nb_train:]

In [241]:
def entropy(data:np.array) -> float:
    """
    Calculates the entropy given the following formula: 
    H(x) = -sum_j(p_j * log(p_j))
    """
    ## get the unique counts of the array 
    _, counts = np.unique(data, return_counts=True)
    ## get the probabilities
    probas = counts / data.shape[0]
    if np.sum(probas) == 0:
        return 0 
    elif np.sum(probas) == 1:
        return 0
    else:
        probas = probas + 1e-7
        ## add the 1e-7 to avoid log(0)
        return np.sum(-probas * np.log2(probas))/float(data.shape[0])
    
def information_gain(left_branch, right_branch) -> float:
    """
    Calculates the information gain given the following formula:
    G(x) = H(x) - sum_j(p_j * H(x|j))
    """
    p = len(left_branch) / (len(left_branch) + len(right_branch))
    return entropy(left_branch) + entropy(right_branch) - p * entropy(left_branch) - (1 - p) * entropy(right_branch)

In [254]:
def get_bootstrap_samples(X_train:np.array, y_train:np.array) -> np.array:
    """
    Function which gets the bootstrap samples.
    These are samples WITH replacement, thus, the left over samples are 
    Left_Over = lim_x_to_inf [(1 - 1/n)^n] -> e^-1 ~ 1/3
    So for the fitting 2/3 of the observations will be used. 
    
    PARAMS:
    -------
    X_train : training samples of the dataset 
    
    y_train : training labels of the dataset 
    
    RETURNS:
    --------
    X_bootstrap : bootstrap samples for X_train 
    
    y_bootstrap : bootstrap samples for y_train 
    
    x_oob : out of bag samples for X_train 
    
    y_oob : out of bag samples for y_train 
    
    """
    ## first get the indices of the bootstrap samples 
    bootstrap_idx = np.random.choice(range(X_train.shape[0]), size=(X_train.shape[0]), replace=True)
    ## get the bootstrap samples
    x_bootstrap = X_train.loc[X_train.index.intersection(bootstrap_idx)].values
    y_bootstrap = y_train[y_train.index.intersection(bootstrap_idx)]
    return x_bootstrap, y_bootstrap

In [285]:
## define the gini function 
def gini(data: np.array) -> float:
    """
    Calculates the gini impurity given the following formula:
    G(x) = 1 - sum_j(p_j^2)
    """
    ## get the unique counts of the array 
    _, counts = np.unique(data, return_counts=True)
    ## get the probabilities
    probas = counts / data.shape[0]
    return 1 - np.sum(probas ** 2)

## define a function to calculate the OOB error
def out_of_bag_error(x_test:np.array, y_test:np.array, model:object) -> float:
    """
    Calculates the out of bag error given the following formula:
    E_OOB = 1/n * sum_i(1 - p(x_i|y_i))
    """
    ## get the predictions for each item in the test set
    preds = np.array([predict_samples(model, i) for i in x_test])
    ## get the sum of the mismatches 
    mismatch = np.sum(preds != y_test)
    ## return the error
    return mismatch / y_test.shape[0]

## define a function to find the best split point 
## it should select m features at random 
## for each feature in the bootstrapped samples, it calculates the information gain
## returns a dictionary with: feature index, split value, left_branch, right_branch
def best_split_finder(X_bootstrap: np.array, y_bootstrap: np.array, max_features_to_use: int) -> dict:
    """
    Calculates the best split for each feature in the bootstrapped samples.
    
    PARAMS: 
    -------
    X_bootstrap : training bootstrapped samples 
    
    y_bootstrap : training labels boostrapped samples
    
    max_features_to_use : (F) the number of features to consider when doing the splits
    
    RETURNS:
    -------
    best_split : dictionary with the index of the feature, the split value, and the right & left nodes
    
    """
    ## get the number of features which are in the bootstrap samples 
    #n_features = X_bootstrap.shape[1]
    n_features = len(X_bootstrap[0])
    ## n_features is the the length of the bootstrap if there are two dimensions else its the length of the first dimension
    ## keep a holder for the features which were used 
    feature_holder = np.array([],dtype=np.float32)
    ## while the length of the feature holder is less than the max features to use
    ## keep adding features to the feature holder
    ## this is to ensure that we don't use the same feature twice
    while len(feature_holder) <= max_features_to_use:
        feature_to_add = np.random.choice(range(n_features)) ## defaults to 1
        ## check if the feature is not in the holder 
        if feature_to_add not in feature_holder:
            #feature_holder.append(feature_to_add)
            ## add it to the array 
            feature_holder = np.append(feature_holder, feature_to_add)
    ## start calculating the information gain for each feature 
    ## keep track of the best information gain & the current node 
    top_information_gain = -999999
    feat_idx = None
    left_child_x,right_child_x = None, None
    left_child_y,right_child_y = None, None
    ## iterate over the features at their indices
    for f_idx in feature_holder:
        f_idx = int(f_idx)
        for best_split_point in X_bootstrap[:, f_idx]:
            ## define the children & their bootstrap 
            right_child_x_bootstrap, right_child_y_bootstrap = np.array([],dtype=np.float32),np.array([],dtype=np.float32)
            left_child_x_bootstrap,left_child_y_bootstrap = np.array([],dtype=np.float32),np.array([],dtype=np.float32)
            ## enumerate the bootstrap samples
            for split_idx, boot_val in enumerate(X_bootstrap[:, f_idx]):
                ## check if the value is less than the split point
                if boot_val <= best_split_point:
                    ## add it to the left child bootstrap
                    left_child_x_bootstrap = np.append(left_child_x_bootstrap, X_bootstrap[split_idx])
                    left_child_y_bootstrap = np.append(left_child_y_bootstrap, y_bootstrap.iloc[split_idx])
                else:
                    ## add it to the right child bootstrap
                    right_child_x_bootstrap = np.append(right_child_x_bootstrap, X_bootstrap[split_idx])
                    right_child_y_bootstrap = np.append(right_child_y_bootstrap, y_bootstrap.iloc[split_idx])
            ## calculate the information gain for the current split
            curr_split_gain = information_gain(left_child_y_bootstrap, right_child_y_bootstrap)
            ## check if the current split is better than the current node
            if curr_split_gain > top_information_gain:
                ## update the top information gain
                top_information_gain = curr_split_gain
                ## update the current node
                # curr_node = {
                #     'information_gain': top_information_gain,
                #     'feature_idx': f_idx,
                #     'split_val': best_split_point,
                #     'left_child': left_child_x_bootstrap,
                #     'right_child': right_child_x_bootstrap
                #              }
                feat_idx = f_idx
                left_child_x = left_child_x_bootstrap
                right_child_x = right_child_x_bootstrap
                left_child_y = left_child_y_bootstrap
                right_child_y = right_child_y_bootstrap
    
    node_dict = {
                'information_gain': top_information_gain,
                'feature_idx': feat_idx,
                'split_val': best_split_point,
                'left_child': {"X_bootstrap":left_child_x, "y_bootstrap":left_child_y},
                'right_child': {"X_bootstrap":right_child_x, "y_bootstrap":right_child_y}
                            }
    
    return node_dict 
    

In [286]:
x_b, y_b = get_bootstrap_samples(X_train, y_train)

In [287]:
node = best_split_finder(x_b, y_b, max_features_to_use=5)

In [288]:
## define function for a terminal node 
def terminal_node(node:dict) -> int:
    ## get the y bootstrap 
    y_boot = node['y_bootstrap']
    return max(y_boot, key=y_boot.count)

## define a function to split the node 
def node_splitted(node: dict,
                  max_features:int,
                  min_samples_split:int,
                  max_depth:int,
                  depth:int) -> None:
    """
    Checks if the node is splitted or not.
    
    PARAMS:
    -------
    node : the node to check if it is splitted or not
    
    max_features : the number of features to consider when doing the splits
    
    min_samples_split : the minimum number of samples to split a node
    
    max_depth : the maximum depth to split a node
    
    depth : the current depth of the node
    
    RETURNS:
    --------
    None : splits node of the tree 
    
    """
    ## get the children 
    left, right = node['left_child'], node['right_child']
    ## delete the nodes 
    del(node['left_child'])
    del(node['right_child'])
    
    ## check if the bootstrap is empty, for either child 
    if len(left['y_bootstrap']) == 0 or len(right['y_bootstrap']) == 0:
        ## create the empty node, which is equal to the sum of the children's y_boot
        empty_child_node = {"y_bootstrap":left['y_bootstrap'] + right['y_bootstrap']}
        ## add the empty node to the left & right child
        node['left_child'] = terminal_node(empty_child_node)
        node['right_child'] = terminal_node(empty_child_node)
        return 
    ## check if the depth is greater than the max depth
    if depth >= max_depth:
        ## set the node to a terminal node 
        node['left_child'] = terminal_node(left)
        node['right_child'] = terminal_node(right)
        return node
    ## check if the number of samples is less than the min samples split
    if len(left['X_bootstrap']) <= min_samples_split:
        ## set the node to a terminal node 
        node['split_left'] = node['split_right'] = terminal_node(left)
    else:
        #return left['X_bootstrap'], left['y_bootstrap']
        ## find the split point for the node
        node['split_left'] = best_split_finder(left['X_bootstrap'], left['y_bootstrap'], max_features_to_use=max_features)
        ## split the nodes 
        node_splitted(node['split_left'], max_features, min_samples_split, max_depth, depth+1)
    ## the same for the right node 
    if len(right['X_bootstrap']) <= min_samples_split:
        ## set the node to a terminal node 
        node['split_right'] = node['split_left'] = terminal_node(left)
    else:
        ## find the split point for the node 
        node['split_right'] = best_split_finder(right['X_bootstrap'], right['y_bootstrap'], max_features_to_use=max_features)
        ## split the nodes 
        node_splitted(node['split_right'], max_features, min_samples_split, max_depth, depth+1)

In [289]:
import copy
n_c = copy.copy(node)
node_splitted(n_c, max_features=1,
              min_samples_split=2,
              max_depth=10,
              depth=1)

TypeError: object of type 'numpy.float64' has no len()